In [105]:
import pickle
import pandas as pd
from glob import glob
from pandas import Index
import sys
sys._enablelegacywindowsfsencoding()
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [99]:
def signal_process(table,event_var='PM25_Marseille_Longchamp',event_threshold = 40, duration = 6,var_name_list='air pollution'):
    """Find the spike in pm25 and plot a sliding window for google trend """

    index_list=np.where(table[event_var]>event_threshold)[0]
    
    # add first element
    shrink_index_list = [index_list[0]]
    date_set ={table.date[index_list[0]]}
    
    # exclude for the same day
    for cnt in range(1,len(index_list)):
        if table.date[index_list[cnt]] in date_set:
            pass
        else:
            shrink_index_list.append(index_list[cnt].copy())
            date_set.add(table.date[index_list[cnt]])
    
    bind_list = []
    for start_index in shrink_index_list:
        this_sequence = table.loc[range(start_index-2,start_index+duration),
                                  var_name_list]
        start_date = table.date[start_index]
        tuples = list(zip([start_date]*(duration+2),range(-2,duration)))

        this_sequence.index = pd.MultiIndex.from_tuples(tuples,names=['event_date_index','hour_index'])
    #         this_sequence = (this_sequence- this_sequence.mean())/this_sequence.std()

        bind_list.append(this_sequence)
    new_table=pd.concat(bind_list)     
    
    return new_table

In [48]:
def get_table(path="..\\stage\\FR-J\\*"):
    df_list =[]
    for filepath in glob(path):
        with open(filepath,'br') as f:
            this_df = pickle.load(f).reset_index()
            col_name = list(this_df.columns)
            col_name[2]=col_name[1]+"_topic"
            this_df.columns =Index(col_name,dtype='object')
            df_list.append(this_df)
    merged_t = df_list[0]
    for i in range(1,len(df_list)):
        merged_t=merged_t.merge(df_list[i],how='outer',on='date')
    return merged_t

In [65]:
get_table(path='..\\stage\\FR-U\\*').head().date

0   2017-08-31 00:00:00
1   2017-08-31 01:00:00
2   2017-08-31 02:00:00
3   2017-08-31 03:00:00
4   2017-08-31 04:00:00
Name: date, dtype: datetime64[ns]

In [63]:
parca=pd.read_csv('../stage/difficultés respiratoires_asthme_dyspnée_respiration_PM25_Marseille_Longchamp.csv')

In [71]:
parca['date']=parca.date.apply(lambda string: datetime.strptime(string,'%Y-%m-%d %H:%M:%S') )

In [73]:
parca_merged=parca.merge(get_table(path='..\\stage\\FR-U\\*'),on='date')

In [ ]:
parca_merged=parca_merged.drop(columns=['Unnamed: 0','datetime','isPartial_x','isPartial_x','isPartial_y'])

In [95]:
parca_merged.columns=Index(['PM25_Marseille_Longchamp', 'date', 'difficultés respiratoires',
       'asthme', 'dyspnée', 'respiration', 'banane', 'banane_topic',
       'la diarrhée', 'la diarrhée_topic', 'pollution de l’air',
       'pollution de l’air_topic'],
      dtype='object')

In [96]:
my_list =list(parca_merged.columns)
my_list.remove('date')

In [97]:
my_list

['PM25_Marseille_Longchamp',
 'difficultés respiratoires',
 'asthme',
 'dyspnée',
 'respiration',
 'banane',
 'banane_topic',
 'la diarrhée',
 'la diarrhée_topic',
 'pollution de l’air',
 'pollution de l’air_topic']

In [109]:
parca_signal=signal_process(parca_merged,event_var='PM25_Marseille_Longchamp',duration=12,event_threshold=35,var_name_list=my_list)

In [150]:
parca_signal.to_csv("../output/air_parca_threshold_35_colsum_duration_12H.csv")

In [ ]:
# parif 

In [113]:
parif_com=get_table(path='..\\stage\\FR-J\\*')

In [116]:
parif_table = pd.read_csv("../stage/parif_asthme_dyspnée_respiration_pollution de l’air_Masque_Purificateur d'air_PA04C_PM10.csv")

In [120]:
parif_table['date']=parca.datetime.apply(lambda string: datetime.strptime(string,'%Y-%m-%d %H:%M:%S'))
parif_table['datetime']=parif_table['date']

In [122]:
parif_merged=parif_table.merge(parif_com,on='date')

In [125]:
parif_merged=parif_merged[['PA04C_PM25','datetime', 'date', 'asthme', 'asthme topic', 'dyspnée',
       'dyspnée topic', 'respiration', 'respiration topic',
       'pollution de l’air', 'pollution de l’air topic', 
       'Masque topic', 'la diarrhée',
       'la diarrhée_topic',
       'banane', 'banane_topic',]]

In [128]:
parif_list = list(parif_merged.columns)
parif_list.remove('datetime')
parif_list.remove('date')

In [129]:
parif_signal=signal_process(parif_merged,event_var='PA04C_PM25',duration=12,event_threshold=35,var_name_list=parif_list)

In [151]:
parif_signal.to_csv("../output/air_parif_threshold_35_colsum_duration_12H.csv")

In [ ]:
# limoges

In [131]:
Limoges_table = pd.read_csv("../stage/limoges.csv").drop(columns=['Year','Month','Day','UTC Hour'])
Limoges_table.columns=Index(['PM25_Limoges', 'PM10_Limoges', 'datetime'], dtype='object')

In [134]:
def conv(datestring):
    try :
        val = datetime.strptime(datestring ,'%Y-%m-%d %H:%M:%S')
    except TypeError:
        val = datestring
    return val

In [135]:
Limoges_table['datetime'] = Limoges_table['datetime'].apply(conv)

In [136]:
Limoge_com=get_table(path="..\\stage\\FR-B\\*")

In [143]:
Limoge_com=Limoge_com.drop(columns=['isPartial_x','isPartial_y','isPartial'])

In [145]:
Limoges_merged=Limoges_table.merge(Limoge_com,left_on='datetime',right_on='date')

In [147]:
Limoges_list =list(Limoges_merged.columns)
Limoges_list.remove("date")
Limoges_list.remove("datetime")

In [148]:
Limoges_signal=signal_process(Limoges_merged,event_var='PM25_Limoges',duration=12,event_threshold=35,var_name_list=Limoges_list)

In [152]:
Limoges_signal.to_csv("../output/air_limoges_threshold_35_colsum_duration_12H.csv")